In [1]:
import tensorflow as tf

tf.config.optimizer.set_jit(True)

In [2]:
import tensorflow_model_optimization as tfmot

In [3]:
import chess

In [4]:
pieces2 = {"r" :-5,
           "R": 5,
           "n" : -3,
           "N": 3,
           "b" : -3,
           "B": 3,
           "p": -1,
           "P": 1,
           "q": -9,
           "Q": 9,
           "k": 0,
           "K": 0
}
def quickeval(boardd):
    fen= boardd.fen()
    if(boardd.outcome(claim_draw=True) is not None):
        if(boardd.outcome(claim_draw=True).result()=="1-0"):
            return 1000
        if(boardd.outcome(claim_draw=True).result()=="0-1"):
            return -1000
        if(boardd.outcome(claim_draw=True).result()=="1/2-1/2"):
            return 0
    parts = fen.split()
    board = parts[0]
    rows = board.split("/")
    board2 =[]
    for f in rows:
        g=[]
        n= list(f)
        for s in n:
            if s in pieces2:
                g.append(pieces2[s])
            else:
                j=int(s)
                d=0
                while d < j:
                    g.append(0)
                    d = d+1
        board2.extend(g)
    return sum(board2)

In [5]:
pieces = {"r" :-4,
           "R": 4,
           "n" : -2,
           "N": 2,
           "b" : -3,
           "B": 3,
           "p": -1,
           "P": 1,
           "q": -5,
           "Q": 5,
           "k": -6,
           "K": 6
}

diction = {-6:0,6:1,-5:2,5:3,-4:4,4:5,-3:6,3:7,-2:8,2:9,-1:10,1:11}
def evaluate(boardd):
    fen = boardd.fen()
    if(boardd.outcome(claim_draw=True) is not None):
        if(boardd.outcome(claim_draw=True).result()=="1-0"):
            return 1000
        if(boardd.outcome(claim_draw=True).result()=="0-1"):
            return -1000
        if(boardd.outcome(claim_draw=True).result()=="1/2-1/2"):
            return 0
    parts = fen.split()
    board = parts[0]
    rows = board.split("/")
    board2 =[]
    for f in rows:
        g=[]
        n= list(f)
        for s in n:
            if s in pieces:
                g.append(pieces[s])
            else:
                j=int(s)
                d=0
                while d < j:
                    g.append(0)
                    d = d+1
        board2.extend(g)
    board = convert(board2)
    preds = chess_model2.predict(board.reshape(1,8,8,12))
    prediction = scaler.inverse_transform(preds).reshape(-1)[0]
    return prediction+0.6*quickeval(boardd)
def convert (inboard):
    board = np.zeros((8,8,12))
    for i in range(64):
        if inboard[i]!=0:
            board[int(i/8)][i%8][diction[inboard[i]]]=1
            board[int(i/8)][i%8][diction[-1*inboard[i]]]=-1
    return board

In [6]:
chess_model2 = tf.keras.models.load_model('chessmodelOptimize.hdf5')

In [7]:
import pickle

In [8]:
file = open('scaler.pickle', 'rb')

In [9]:
scaler=pickle.load(file)
file.close()

In [15]:
meow= chess.Board()

In [17]:
import numpy as np

In [20]:
%timeit evaluate(meow)

30.5 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
converter=tf.lite.TocoConverter.from_saved_model('chessmodelOptimize.hdf5')
converter.post_training_quantize=True
tflite_quantized_model=converter.convert()
open("quantized_model.tflite", "wb").write(tflite_quantized_model)

AttributeError: module 'tensorflow._api.v2.lite' has no attribute 'TocoConverter'

In [29]:
converter = tf.lite.TFLiteConverter.from_keras_model(chess_model2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = 'model.tflite'

with open(quantized_and_pruned_tflite_file, 'wb') as f:
    f.write(quantized_and_pruned_tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ACHYUT~1\AppData\Local\Temp\tmpa0o3hl6a\assets


INFO:tensorflow:Assets written to: C:\Users\ACHYUT~1\AppData\Local\Temp\tmpa0o3hl6a\assets


In [30]:
f.close()

In [ ]:
tf.lite.